# Agentspace Research Assistant - API

## Setup

In [31]:
import json
import requests
import json_stream
from json_stream.base import StreamingJSONList, StreamingJSONObject
import io

from IPython.display import display, Markdown,HTML,Image

In [16]:
raw_token = ! gcloud auth print-access-token
token = f'Bearer {raw_token[0]}'

In [17]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
ENGINE_ID = "agentspace-pe-main_1739458386326"
BASE_URL  = 'https://discoveryengine.googleapis.com/v1alpha'

In [18]:
# URL and headers for Agentspace agent queries
request_url = (
    f'{BASE_URL}'
    f'/projects/{PROJECT_ID}'
    f'/locations/global'
    f'/collections/default_collection'
    f'/engines/{ENGINE_ID}/assistants/default_assistant:streamAssist'
)
headers = {
    'Authorization': token,
    'Content-Type': 'application/json',
    'X-Goog-User-Project': PROJECT_ID
}

## Helper Functions

In [4]:
def extract_plan(json_data):
    texts = []
    session = None
    
    # Process the json_data which is a list
    for item in json_data:
        # Extract session information
        if 'sessionInfo' in item and 'session' in item['sessionInfo']:
            session = item['sessionInfo']['session']
        
        # Navigate to the replies in the answer
        if 'answer' in item and 'replies' in item['answer']:
            replies = item['answer']['replies']
            
            # Iterate through each reply
            for reply in replies:
                # Check if there's groundedContent with text
                if ('groundedContent' in reply and 
                    'content' in reply['groundedContent'] and 
                    'text' in reply['groundedContent']['content']):
                    
                    # Extract the text
                    text = reply['groundedContent']['content']['text']
                    texts.append(text)
    
    # Join all texts with newlines to create markdown
    markdown_text = "\n\n".join(texts)
    
    return markdown_text, session


## Create the Plan

In [19]:
query_plan = "How did DeepSeek impact NVIDIA share prices?"

json_plan = {
    'query': {
        'text': query_plan,
    },
    'answerGenerationMode': 'RESEARCH',
    'googleSearchGroundingEnabled': True
}

In [20]:
response_plan = requests.post(request_url, headers = headers, json = json_plan)
print(response_plan)

<Response [200]>


In [21]:
research_plan, session_id = extract_plan(response_plan.json())
Markdown(research_plan)

I have created a research plan to investigate the impact of DeepSeek on NVIDIA's share prices. Please review it and let me know if you have any suggestions or modifications.

## Impact of DeepSeek on NVIDIA Share Prices

### **Understanding DeepSeek**
Research DeepSeek to understand its products, services, and its position in the market, especially in relation to AI and computing.

### **Overview of NVIDIA's Business and Stock Performance**
Gather information on NVIDIA's core business, recent stock performance, and key market factors influencing its valuation.

### **DeepSeek and NVIDIA: Competitive Analysis**
Analyze the competitive landscape between DeepSeek and NVIDIA, focusing on areas of overlap and potential competition in AI, hardware, or software.

### **Market Sentiment Analysis Regarding DeepSeek and NVIDIA**
Investigate news articles, financial analysis reports, and online discussions to gauge market sentiment towards both companies.

### **Correlation Analysis of DeepSeek's Announcements and NVIDIA's Stock Price**
Examine specific announcements, product launches, or partnerships involving DeepSeek and analyze their correlation with changes in NVIDIA's stock price.

### **Expert Opinions and Financial Analysis**
Consult expert opinions from financial analysts and industry experts regarding the potential impact of DeepSeek on NVIDIA's market position and stock valuation.

## Run the Plan (*non-stream*)

In [22]:
query_research = "Start research"

json_research = {
    "query": {
        "text": query_research,
    },
    "session": session_id,
    "answerGenerationMode": "RESEARCH"
}

In [23]:
response_research = requests.post(request_url, headers = headers, json = json_research)
print(response_research)

<Response [200]>


In [30]:
response_research.json()[555]['answer']['replies'][0]

{'groundedContent': {'content': {'role': 'model',
   'text': ' of the connected data sources.'},
  'textGroundingMetadata': {},
  'contentMetadata': {'contentKind': 'RESEARCH_ANSWER', 'contentId': 'ID14'}}}

In [10]:
report = []
for i in range(len(response_plan.json())):
    if response_plan.json()[i]['answer']['state'] != "SUCCEEDED":
        if 'contentKind' in response_plan.json()[i]['answer']['replies'][0]['groundedContent']['contentMetadata']:
            if response_plan.json()[i]['answer']['replies'][0]['groundedContent']['contentMetadata']['contentKind'] == 'RESEARCH_REPORT':
                report.append(response_plan.json()[i]['answer']['replies'][0]['groundedContent']['content']['text'])
    else:
        print("All done!")

All done!


In [11]:
report_str = ' '.join(report)
Markdown(report_str)

#  Research Document: Impact of DeepSeek on NVIDIA Share Prices

This document analyzes the  potential impact of DeepSeek, a Chinese AI company, on NVIDIA's share  prices.  We will examine DeepSeek's offerings, NVIDIA's market position, the overlap between their businesses, and the market's reaction to Deep Seek's advancements.

## 1. Understanding DeepSeek

DeepSeek is a Chinese artificial intelligence company based in Hangzhou, Zhejiang, founded in July  2023 by Liang Wenfeng .  Wenfeng also co-founded High-Flyer, a Chinese hedge fund that owns and funds DeepSeek, and serves as CEO for both companies  . DeepSeek focuses on developing open-source large language models (LLMs) , offering a range of products and services centered around AI and data analysis .

###  1.1 DeepSeek's Products and Services

DeepSeek's main offerings include:

*   **AI Models:**  A series of open-source LLMs such as DeepSeek Coder, DeepSeek LLM, DeepSeek-V2, DeepSeek V3, and DeepSeek R1,  designed for various tasks including coding, general-purpose applications, and reasoning .  DeepSeek-R1, released in January 2025 under the MIT License, is particularly noteworthy for its performance and open-source nature .   DeepSeek also released Janus-Pro-7B, a vision model in January 2025 .
*   **DeepSeek AI Assistant:** A mobile application providing a chatbot interface for DeepSeek-R1 .
*   **API Access:** An API allowing developers to integrate DeepSeek 's AI models into their projects .
*   **DeepSeek Platform:** A developer platform .
*   **Data Analysis and Information Retrieval:** An advanced platform for data analytics and information retrieval using AI technologies .
*   **Services:**  Services such as big data analysis, natural language processing, and AI-powered algorithms for businesses .

DeepSeek gained international attention for allegedly matching the performance of leading Western AI models at a fraction of the cost , positioning itself as an open-source alternative to ChatGPT .

##  2. NVIDIA's Market Position and Influencing Factors

As of March 2025, NVIDIA holds a dominant market position, especially in GPUs for high-performance gaming and data center AI workloads .  It boasts a market capitalization of $2.916 trillion USD, making it  the world's second most valuable company , with a market share of 44.85% .  Its recent quarterly revenue reached $39.33B USD, exceeding estimates .  Analysts predict a stock price range of $130.00  to $235.92 USD .

Several factors influence NVIDIA's share price:

*   **AI Market Trends:**  Demand for generative AI and supercomputers significantly impacts NVIDIA's revenue, as its GPUs are crucial for machine learning and generative AI models  .
*   **Financial Performance:** Revenue growth and earnings reports directly affect the stock price .
*   **Competition:** Competition from AMD and Intel influences NVIDIA's market share and stock price .
*   **Market  Capitalization:** Changes in market capitalization reflect investor sentiment and influence the stock price .
*   **Chip Export Restrictions:** Restrictions on chip exports, particularly to China, can negatively impact revenue and stock price .
*   **Investor Sentiment and Valuation:**  Investor confidence and valuation significantly impact the stock price .
*   **Cryptocurrency Market:** Fluctuations in the cryptocurrency market can affect GPU demand for mining, impacting the stock price .
*   **Economic Factors:** Economic uncertainty and tariffs can lead to market sell-offs  .
*   **Company-Specific Developments:** New chip transitions and announcements at AI conferences can influence investor interest .
*   **Demand for Data Centers:** Increasing demand for processing power in data centers drives NVIDIA's market leadership .
*   **Gaming Industry:** NVIDIA  maintains a strong position in the gaming industry .


## 3. DeepSeek and NVIDIA: Overlap and Interaction

The primary overlap between DeepSeek and NVIDIA lies in AI and high-performance computing . DeepSeek utilizes LLMs for generative AI services  , while NVIDIA provides the GPUs and CUDA platform, a standard for AI and machine learning workloads .  NVIDIA's hardware and software are used to develop and deploy AI models like those created by DeepSeek  .

NVIDIA has publicly commented on DeepSeek, praising its advancements and highlighting DeepSeek-R1's availability as an NVIDIA NIM microservice .  They emphasize the importance of NVIDIA GPUs and high-performance networking for AI inference  and the benefits of their Blackwell  architecture for DeepSeek-R1's performance .  They also highlight the DeepSeek R1 FP4 model, a quantized version of DeepSeek R1 .

## 4. Market Reaction to DeepSeek

The release of DeepSeek's AI models, particularly DeepSeek -R1 on January 10, 2025 , caused a significant market reaction .  NVIDIA's stock price experienced a substantial drop on January 27, 2025, its largest one-day decline since  March 2020 , erasing a significant portion of its market capitalization . This sell-off impacted the broader tech market .  Investor concerns stemmed from DeepSeek' s ability to achieve comparable performance to leading models with less powerful hardware , questioning the necessity of massive spending on AI systems .  DeepSeek's software-driven optimization approach, potentially reducing demand for high-end chips, contributed to the market's  reaction .  While some analysts viewed the drop as an overreaction , others suggested that DeepSeek's success could shift the industry focus towards efficiency .

Financial news articles and analyst reports highlighted the market impact of DeepSeek's cost-efficient AI model  , noting the significant decline in AI-related company stock prices, including NVIDIA's .  Analysts offered varied perspectives, some seeing a buying opportunity  , while others questioned DeepSeek's claims .  The long-term implications for NVIDIA and the broader AI sector remain uncertain .

Major investors and analysts expressed diverse opinions . Some saw a buying opportunity due  to potential increased demand for NVIDIA GPUs , while others expressed skepticism about DeepSeek's claims and potential impact on GPU demand .  NVIDIA itself maintained a positive view of DeepSeek's achievements . 

## 5.  NVIDIA's Stock Price Around DeepSeek Model Releases

The release of DeepSeek's R1 model on January 10, 2025  coincided with a significant drop in NVIDIA's stock price on January 27,  2025 , closing at $126.75, an 11.13% drop .  Some sources report a 17% decline  , resulting in a market capitalization decrease of approximately $589 billion .  The stock recovered somewhat in the following weeks .  Information on NVIDIA's stock prices around the release dates of other DeepSeek models is limited.  However, we know  that on May 6, 2024, the closing price was $92.13 , on July 18, 2024, it was $121.07 , and on December 20, 2 024, it was $134.70 .  Finally, in November 2023, the closing price was $46.75 .

## 6. NVIDIA 's Q1 2025 Earnings

NVIDIA's Q1 2025 earnings report showed strong performance .  Total revenue exceeded estimates at $26.0 billion , with Data Center revenue significantly surpassing expectations at $22.6 billion  .  Gaming revenue also increased .  The report highlighted strong demand for Hopper GPUs and provided positive guidance for Q2 2025 .

## 7. Executive Summary

DeepSeek's emergence as a provider of cost -effective, open-source AI models has created a ripple effect in the AI industry and financial markets.  While DeepSeek's technology presents a potential challenge to NVIDIA's dominance in the high-end GPU market, the market's initial reaction, characterized by a significant drop in NVIDIA's stock price following  the release of DeepSeek-R1, appears to have been an overreaction by some accounts.  NVIDIA's strong Q1 2025 earnings, however, demonstrate the company's continued success and resilience.  The long-term impact of DeepSeek on NVIDIA's business remains to be seen , but the situation highlights the dynamic and competitive nature of the AI landscape.  Further research is needed to fully understand the evolving relationship between these two companies and its impact on the broader AI market.
 

## Run the Plan (*stream*) - WIP

In [41]:
query_research = "Start research"

json_research = {
    "query": {
        "text": query_research,
    },
    "session": session_id,
    "answerGenerationMode": "RESEARCH"
}

In [40]:
def stream_text_response_lines(request_url, headers, json):
    """
    Performs a POST request and yields the response as a stream of text lines.

    Args:
        request_url (str): The URL to send the POST request to.
        headers (dict): Headers to include in the request.
        json_plan (dict): The JSON data to send in the request body.

    Yields:
        str: The next line of text from the response.
             Yields an error message string if the request fails.
    """
    try:
        with requests.post(request_url, headers=headers, json=json, stream=True) as response:
            response.raise_for_status()
            # Yield each line as text, decoding using the response's encoding
            for line in response.iter_lines(decode_unicode=True):
                yield line

    except Exception as e:
        yield f"An error occurred: {e}"

def stream_text_response_chunks(request_url, headers, json, chunk_size=1024):
    """
    Performs a POST request and yields the response as a stream of text chunks.

    Args:
        request_url (str): The URL to send the POST request to.
        headers (dict): Headers to include in the request.
        json_plan (dict): The JSON data to send in the request body.
        chunk_size (int): The size of the chunks to read in bytes.

    Yields:
        str: The next chunk of text from the response.
             Yields an error message string if the request fails.
    """
    try:
        with requests.post(request_url, headers=headers, json=json, stream=True) as response:
            response.raise_for_status()
            # Yield chunks of text, decoding using the response's encoding
            for chunk in response.iter_content(chunk_size=chunk_size, decode_unicode=True):
                yield chunk

    except Exception as e:
        yield f"An error occurred: {e}"


In [42]:
print("--- Streaming text lines ---")
text_stream_lines = stream_text_response_lines(request_url, headers, json_research)
if text_stream_lines:
    for line in text_stream_lines:
        print(f"Received line: {line}")

    

--- Streaming text lines ---
Received line: An error occurred: 401 Client Error: Unauthorized for url: https://discoveryengine.googleapis.com/v1alpha/projects/mg-ce-demos/locations/global/collections/default_collection/engines/agentspace-pe-main_1739458386326/assistants/default_assistant:streamAssist


In [ ]:
print("\n--- Streaming text chunks (128 bytes) ---")
text_stream_chunks = stream_text_response_chunks(request_url, headers, json_plan, chunk_size=128)
if text_stream_chunks:
    for chunk in text_stream_chunks:
        print(f"Received chunk: {chunk!r}") # Using !r to show raw string with potential newlines


In [ ]:
report = []
for i in range(len(response_plan.json())):
    if response_plan.json()[i]['answer']['state'] != "SUCCEEDED":
        if 'contentKind' in response_plan.json()[i]['answer']['replies'][0]['groundedContent']['contentMetadata']:
            if response_plan.json()[i]['answer']['replies'][0]['groundedContent']['contentMetadata']['contentKind'] == 'RESEARCH_REPORT':
                report.append(response_plan.json()[i]['answer']['replies'][0]['groundedContent']['content']['text'])
    else:
        print("All done!")